In [4]:
# 라이브러리 모음
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.graph_objects as go
import plotly.express as px

# 모듈 모음
import modules as mds

In [5]:
# Windows에서 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # '맑은 고딕'이 설치되어 있을 경우
plt.rcParams['axes.unicode_minus'] = False     # 마이너스(-) 부호 깨짐 방지

In [ ]:
# macOS에서 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'   # macOS 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 누적 설비용량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total = pd.read_excel("dataset/지역별_누적_설비용량.xlsx")
data_2024 = pd.read_excel("dataset/2024년도_신규_설비용량.xlsx")
data_2023 = pd.read_excel("dataset/2023년도_신규_설비용량.xlsx")
data_2022 = pd.read_excel("dataset/2022년도_신규_설비용량.xlsx")
data_2021 = pd.read_excel("dataset/2021년도_신규_설비용량.xlsx")

# 데이터프레임으로 변환
# ic = Installed Capacity = 설비용량
ic_total = pd.DataFrame(data_total)
ic_2024 = pd.DataFrame(data_2024)
ic_2023 = pd.DataFrame(data_2023)
ic_2022 = pd.DataFrame(data_2022)
ic_2021 = pd.DataFrame(data_2021)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(ic_total)
mds.delete_comma(ic_2024)
mds.delete_comma(ic_2023)
mds.delete_comma(ic_2022)
mds.delete_comma(ic_2021)

# 컴럼 통일화를 위해 capacity_total에 "합계" 컬럼 추가
ic_total["합계"] = ic_total.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
ic_total = pd.concat([ic_total, ic_2024, ic_2023, ic_2022, ic_2021], ignore_index=True)

# 각 연도별 누적값 계산
for i in range(1, len(ic_total)):
    ic_total.iloc[i, 1:] = ic_total.iloc[i-1, 1:] - ic_total.iloc[i, 1:]

# 연도 정리
ic_total.rename(columns={"Unnamed: 0": "연도"}, inplace=True)
ic_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 2020-2024 각 연도의 지역별 누적 설비용량
ic_total

# 엑셀 파일 저장
# ic_total.to_excel("dataset/지역별_연도별_설비용량.xlsx", index=False, engine="openpyxl")

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,50,275,420,163,353,65,152,88,2012,1613,3772,6023,3842,2016,621,1943,4590,27998
1,2023,50,234,339,139,303,55,122,78,1672,1408,3072,5410,3324,1745,610,1760,4304,24625
2,2022,48,202,297,124,274,50,99,70,1459,1240,2798,4568,2886,1564,587,1612,3995,21873
3,2021,44,170,265,110,249,46,87,62,1278,1065,2522,3911,2409,1347,549,1408,3357,18879
4,2020,41,144,200,100,215,39,69,55,1056,846,2038,3054,1771,1085,425,1105,2559,14802


In [ ]:
# 누적 발전량 데이터 정리

# 누적 / 연도별 데이터 불러오기
data_total_2022 = pd.read_excel("dataset/지역별_누적_발전량.xlsx")
data_2024 = pd.read_excel("dataset/2024년_신규_발전량.xlsx")
data_2023 = pd.read_excel("dataset/2023년_신규_발전량.xlsx")
data_2022 = pd.read_excel("dataset/2022년_신규_발전량.xlsx")
data_2021 = pd.read_excel("dataset/2021년_신규_발전량.xlsx")
data_2020 = pd.read_excel("dataset/2020년_신규_발전량.xlsx")

# 데이터프레임으로 변환
# gen = Power Generation = 발전량
gen_total_2022 = pd.DataFrame(data_total_2022)
gen_2024 = pd.DataFrame(data_2024)
gen_2023 = pd.DataFrame(data_2023)
gen_2022 = pd.DataFrame(data_2022)
gen_2021 = pd.DataFrame(data_2021)
gen_2020 = pd.DataFrame(data_2020)

# 열과 행 전치
gen_2024 = gen_2024.T.reset_index(drop=True)
gen_2023 = gen_2023.T.reset_index(drop=True)
gen_2022 = gen_2022.T.reset_index(drop=True)
gen_2021 = gen_2021.T.reset_index(drop=True)
gen_2020 = gen_2020.T.reset_index(drop=True)

# 컴럼 정리
gen_2024.columns = gen_2024.iloc[0]
gen_2024.drop(labels=gen_2024.index[0], axis=0, inplace=True)
gen_2023.columns = gen_2023.iloc[0]
gen_2023.drop(labels=gen_2023.index[0], axis=0, inplace=True)
gen_2022.columns = gen_2022.iloc[0]
gen_2022.drop(labels=gen_2022.index[0], axis=0, inplace=True)
gen_2021.columns = gen_2021.iloc[0]
gen_2021.drop(labels=gen_2021.index[0], axis=0, inplace=True)
gen_2020.columns = gen_2020.iloc[0]
gen_2020.drop(labels=gen_2020.index[0], axis=0, inplace=True)


# 연도 컬럼 추가
gen_2024.insert(0, "연도", np.nan)
gen_2023.insert(0, "연도", np.nan)
gen_2022.insert(0, "연도", np.nan)
gen_2021.insert(0, "연도", np.nan)
gen_2020.insert(0, "연도", np.nan)

# 숫자의 ',' 때문에 object 값이었던 데이터들을 int 값으로 변환
mds.delete_comma(gen_total_2022)
mds.delete_comma(gen_2024)
mds.delete_comma(gen_2023)
mds.delete_comma(gen_2022)
mds.delete_comma(gen_2021)
mds.delete_comma(gen_2020)

# 월별 데이터 연도별 데이터로 변환
gen_2024 = pd.DataFrame(gen_2024.sum()).T
gen_2023 = pd.DataFrame(gen_2023.sum()).T
gen_2022 = pd.DataFrame(gen_2022.sum()).T
gen_2021 = pd.DataFrame(gen_2021.sum()).T
gen_2020 = pd.DataFrame(gen_2020.sum()).T

# "합계" 컬럼 추가
gen_2024["합계"] = gen_2024.sum(axis=1, numeric_only=True)
gen_2023["합계"] = gen_2023.sum(axis=1, numeric_only=True)
gen_2022["합계"] = gen_2022.sum(axis=1, numeric_only=True)
gen_2021["합계"] = gen_2021.sum(axis=1, numeric_only=True)
gen_2020["합계"] = gen_2020.sum(axis=1, numeric_only=True)

# 하나의 DataFrame으로 병합
gen_total = pd.concat([gen_2024, gen_2023, gen_total_2022, gen_2022, gen_2021], ignore_index=True)

# 연도 정리
gen_total = gen_total.drop(gen_total.columns[-1], axis=1)
gen_total.rename(columns={"지역": "연도"}, inplace=True)
gen_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 소수점 제거
gen_total = gen_total.astype(int)

# # 각 연도별 누적값 계산
gen_total.iloc[1, 1:] = gen_total.iloc[2, 1:] + gen_total.iloc[1, 1:] # 2023 계산
gen_total.iloc[0, 1:] = gen_total.iloc[1, 1:] + gen_total.iloc[0, 1:] # 2024 계산
gen_total.iloc[3, 1:] = gen_total.iloc[2, 1:] - gen_total.iloc[3, 1:] # 2021 계산
gen_total.iloc[4, 1:] = gen_total.iloc[3, 1:] - gen_total.iloc[4, 1:] # 2020 계산

# 2020-2024 각 연도의 지역별 누적 발전량
gen_total.head()

# 엑셀 파일 저장
# gen_total.to_excel("dataset/지역별_연도별_발전량.xlsx", index=False, engine="openpyxl")

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,454992,1584748,1962317,910178,2054262,372182,653470,524858,9209968,8280679,20170041,34738869,19298394,10914932,3712238,11167641,28815406,154825183
1,2023,397620,1296806,1518891,747103,1694113,305236,503506,437543,7143971,6533179,15870905,27619683,14957803,8711121,3077732,8984358,23235897,123035475
2,2022,341328,1056914,1166405,614164,1386507,249449,388527,358492,5478859,4990438,12385263,21236718,11154610,6771133,2429222,6964022,17956708,94928767
3,2021,284628,854401,851754,489619,1087788,196895,282818,287876,4001713,3603271,9087437,15534479,7826973,4971554,1763312,5086129,12875255,69085910
4,2020,232973,687334,598182,375544,837087,155021,195240,224741,2768139,2462795,6310846,10918067,5318784,3562221,1189794,3488999,9022902,48348677


In [ ]:
# 연도별 신규 발전량 데이터 재구성

# 하나의 DataFrame으로 병합
gen_new_total = pd.concat([gen_2024, gen_2023, gen_2022, gen_2021, gen_2020], ignore_index=True).astype(int)
gen_new_total["연도"] = [2024, 2023, 2022, 2021, 2020]

# 2020-2024 연도별 신규 발전량
gen_new_total

# 엑셀 파일 저장
# gen_new_total.to_excel("dataset/지역별_연도별_신규_발전량.xlsx", index=False, engine="openpyxl")

,연도,서울,부산,대구,인천,광주,대전,울산,세종,경기,충북,충남,전남,경북,경남,제주,강원,전북,합계
0,2024,57372,287942,443426,163075,360149,66946,149964,87315,2065997,1747500,4299136,7119186,4340591,2203811,634506,2183283,5579509,31789708
1,2023,56292,239892,352486,132939,307606,55787,114979,79051,1665112,1542741,3485642,6382965,3803193,1939988,648510,2020336,5279189,28106708
2,2022,56700,202513,314651,124545,298719,52554,105709,70616,1477146,1387167,3297826,5702239,3327637,1799579,665910,1877893,5081453,25842857
3,2021,51655,167067,253572,114075,250701,41874,87578,63135,1233574,1140476,2776591,4616412,2508189,1409333,573518,1597130,3852353,20737233
4,2020,47700,135906,191816,100864,225188,36692,68344,57273,969799,871117,2193969,3431766,1819949,1085723,398364,1213860,2881921,15730251


In [ ]:
# 24년 지역별 합계 일사량

solar_radiation_year = pd.read_csv('dataset/지역별_합계_일사량.csv', encoding="cp949")

# 컬럼명 정리
solar_radiation_year.rename(columns={"지점명": "지역"}, inplace=True)

# 지역명 정리
solar_radiation_year["지역"] = ["강원", "서울", "인천", "경기", "충북", "대전", "경북", "대구", "전북", "울산", "광주", "부산", "전남", "제주", "경남", "세종", "충남"]

# 필요한 컬럼으로 재구성
solar_radiation_year = solar_radiation_year.iloc[:, 1:]

# 지역 순서 통일
solar_radiation_year = solar_radiation_year.iloc[[1, 11, 7, 2, 10, 5, 9, 15, 3, 4, 16, 12, 6, 14, 13, 0, 8]].reset_index(drop=True).round()

# 일사량 값 정수로 변환
solar_radiation_year["합계 일사량(MJ/m2)"] = solar_radiation_year["합계 일사량(MJ/m2)"].astype(int)

# 2024 지역별 합계 일사량
solar_radiation_year

# 엑셀 파일 저장
# solar_radiation_year.to_excel("dataset/2024_지역별_합계_일사량.xlsx", index=False, engine="openpyxl")

,지역,일시,합계 일사량(MJ/m2)
0,서울,2024,5236
1,부산,2024,5791
2,대구,2024,5469
3,인천,2024,5492
4,광주,2024,5414
5,대전,2024,5423
6,울산,2024,5458
7,세종,2024,4720
8,경기,2024,5329
9,충북,2024,5220


In [19]:
# 2020-2024 지역별 힙계 일사량

solar_radiation_total_year = pd.read_csv('dataset/20-23_지역별_합계_일사량.csv', encoding="cp949")

# 컬럼명 정리
solar_radiation_total_year.rename(columns={"지점명": "지역"}, inplace=True)
solar_radiation_total_year = solar_radiation_total_year[["지역", "일시", "합계 일사량(MJ/m2)"]]

# 피벗으로 변환
solar_radiation_total_year = solar_radiation_total_year.pivot_table(index="일시", columns="지역", values="합계 일사량(MJ/m2)", aggfunc='max', fill_value=0).round().astype(int)

# 인덱스 컬럼 정리
solar_radiation_total_year.index.name = None
solar_radiation_total_year.columns.name = None
solar_radiation_total_year = solar_radiation_total_year.T

# 지역명 정리
solar_radiation_total_year["지역"] = ["강원", "광주", "대구", "대전", "부산", "서울", "세종", "경기", "경북", "전남", "울산", "인천", "전북", "제주", "경남", "충북", "충남"]
solar_radiation_total_year.reset_index(drop=True, inplace=True)
solar_radiation_total_year = solar_radiation_total_year[["지역", 2020, 2021, 2022, 2023]]

# 지역 순서 통일
solar_radiation_total_year = solar_radiation_total_year.iloc[[5,4,2,11,1,3,10,6,7,15,16,9,8,14,13,0,12]].reset_index(drop=True)

# 24년 데이터 추가
data = solar_radiation_year["합계 일사량(MJ/m2)"]
solar_radiation_total_year[2024] = data

# 평균 일사량 계산
solar_radiation_mean_year = solar_radiation_total_year.copy()
solar_radiation_mean_year.iloc[:,1:] = (solar_radiation_mean_year.iloc[:,1:] / 12).round().astype(int)

# 2020-2024 지역별 합계 일사량
solar_radiation_total_year

# 2020-2024 지역별 평균 일사량
solar_radiation_mean_year

# 합계 일사량 엑셀 파일 저장
# solar_radiation_total_year.to_excel("dataset/2020-2024_지역별_합계_일사량.xlsx", index=False, engine="openpyxl")

# 평균 일사량 엑셀 파일 저장
# solar_radiation_mean_year.to_excel("dataset/2020-2024_지역별_평균_일사량.xlsx", index=False, engine="openpyxl")

,지역,2020,2021,2022,2023,2024
0,서울,426,429,444,432,436
1,부산,474,476,497,475,483
2,대구,463,459,483,456,456
3,인천,430,429,458,444,458
4,광주,460,458,476,458,451
5,대전,460,417,461,451,452
6,울산,0,0,0,441,455
7,세종,0,0,0,424,393
8,경기,420,422,438,438,444
9,충북,448,446,464,449,435


In [ ]:
# 충남 경남 경북 10년치 일사량

area_solar = pd.read_csv("dataset/15-19 충남 경남 경북 합계 일사량.csv", encoding="cp949")

# 20-24 합계 일사량 데이터 충남 경뷱 경남만 복사
area_solar_total_year = solar_radiation_total_year.iloc[[10,12,13],:].reset_index(drop=True)

# 컬럼명 정리 후 사용할 컬럼만 선택
area_solar.rename(columns={"지점명": "지역"}, inplace=True)
area_solar = area_solar[["지역", "일시", "합계 일사량(MJ/m2)"]]

# # 피벗으로 변환
area_solar = area_solar.pivot_table(index="일시", columns="지역", values="합계 일사량(MJ/m2)", aggfunc='max', fill_value=0).round().astype(int)

# # 인덱스 컬럼 정리
area_solar.index.name = None
area_solar.columns.name = None
area_solar = area_solar.T

# 지역 컬럼 추가
area_solar["지역"] = ["충남", "경북", "경남"]
area_solar.reset_index(drop=True, inplace=True)

# 컬럼 순서 정리
area_solar = area_solar[["지역", 2015, 2016, 2017, 2018, 2019]]

# df 결합
area_solar_total_year = pd.concat([area_solar, area_solar_total_year.iloc[:,1:]], axis=1)

# 평균값 구하기
area_solar_total_year.iloc[:,1:] = (area_solar_total_year.iloc[:,1:] / 12).round().astype(int)

# 2015-2024 충남 / 경북 / 경남 연도별 평균 일사량
area_solar_total_year

# excel 파일로 추출
# area_solar_total_year.to_excel("dataset/2015-2024_지역별_평균_일사량.xlsx", index=False, engine="openpyxl")

,지역,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,충남,349,331,343,461,450,430,430,449,430,442
1,경북,441,432,459,483,467,457,447,478,470,463
2,경남,428,289,345,404,465,462,466,484,457,466


In [ ]:
# 24년 지역 / 월별 일사량

solar_radiation_month = pd.read_csv('dataset/지역별_월별_합계_일사량.csv', encoding="cp949")

# 피벗으로 변환
solar_radiation_month = solar_radiation_month.iloc[:, 1:]
solar_radiation_month = solar_radiation_month.pivot_table(index="일시", columns="지점명", values="합계 일사량(MJ/m2)", aggfunc='max', fill_value=0).round().astype(int)

# 인덱스 / 컬럼 정리
solar_radiation_month_df = solar_radiation_month.T.reset_index()
solar_radiation_month_df.index.name = None
solar_radiation_month_df.columns.name = None
solar_radiation_month_df.rename(columns={"지점명": "지역"}, inplace=True)

# 지역명 / 순서 통일
solar_radiation_month_df = solar_radiation_month_df.iloc[[6,5,2,12,1,3,11,7,0,16,4,10,9,15,14,0,13]].reset_index(drop=True)
solar_radiation_month_df["지역"] = ["서울","부산","대구","인천","광주","대전","울산","세종","경기","충북","충남","전남","경북","경남","제주","강원","전북"]

# 결측값 처리
avg = solar_radiation_month_df["2024-01"].mean().round()
avg = avg.astype(int)
solar_radiation_month_df.iloc[10, 1:3] = avg

# 2024 지역별 월별 평균 일사량
solar_radiation_month_df

# 엑셀 파일 저장
# solar_radiation_month_df.to_excel("dataset/2024_지역별_월별_합계_일사량.xlsx", index=False, engine="openpyxl")

,지역,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,서울,280,260,438,558,687,715,402,508,439,367,293,288
1,부산,343,279,508,527,744,594,547,708,516,356,323,348
2,대구,312,258,470,512,724,644,515,634,455,348,293,304
3,인천,271,279,492,561,675,741,455,618,460,374,289,276
4,광주,294,277,472,502,708,630,451,627,479,388,313,274
5,대전,259,258,476,540,721,697,461,613,486,340,308,265
6,울산,309,247,482,499,712,600,565,655,453,316,290,330
7,세종,257,251,463,522,705,680,474,607,480,315,293,262
8,경기,276,205,440,504,650,619,473,566,344,301,248,273
9,충북,238,235,431,529,708,683,466,606,460,335,274,256


In [ ]:
# 경남 태양광 발전소 현황

gn_solar = pd.read_csv("dataset/경상남도_태양광 발전설치현황.csv", encoding="cp949")

# 설비용량 높은 순서대로 정렬
gn_solar.sort_values(["설비용량(킬로와트)"],ascending=False, inplace=True)
gn_solar.reset_index(drop=True, inplace=True)

# 데이터 간소화
gn_solar = gn_solar.head(5)[["상호", "설비용량(킬로와트)", "설치장소"]]

# 컬럼명 통일
gn_solar.rename(columns={"상호": "발전소명", "설비용량(킬로와트)": "설비용량(KW)", "설치장소": "위치"}, inplace=True)

# 위도 / 경도 추가
gn_solar["위도"] = [128.9784927,128.1299150,128.7015250,128.0649578,129.0117573]
gn_solar["경도"] = [35.2816030,35.1150670,35.1921100,35.0774302,35.2925874]

# 경남 발전소 위치 및 용량
gn_solar

# 엑셀 파일 저장
# gn_solar.to_excel("dataset/경남_설비목록.xlsx", index=False, engine="openpyxl")

,발전소명,설비용량(KW),위치,위도,경도
0,세일솔라 주식회사[세일솔라 덕산(3MW) 태양광발전소],3000.00,김해시 대동면 동북로 67,128.978493,35.281603
1,한국농어촌공사 사천지사(두량태양광발전소),2996.91,사천시 사천읍 두량리 94번지 등 13필지,128.129915,35.115067
2,케이에스솔라(주)(GMK 창원공장 태양광발전소),2996.62,창원시 성산구 창원대로 1124 (성주동),128.701525,35.192110
3,코리아에너지발전소(주) 1호기,2993.96,사천시 사남면 공단1로 78,128.064958,35.077430
4,양산솔라태양광발전소,2993.76,양산시 물금읍 제방로 27,129.011757,35.292587


In [ ]:
# 경북 태양광 발전소 현황

gb_solar = pd.read_csv("dataset/경상북도_태양광발전소 현황.csv", encoding="cp949")

# 설비용량 높은 순서대로 정렬
gb_solar.sort_values(["사업개시용량"],ascending=False, inplace=True)
gb_solar.reset_index(drop=True, inplace=True)

# 데이터 간소화
gb_solar = gb_solar.head(5)[["발전소명", "사업개시용량", "설치장소"]]

# 컬럼명 통일
gb_solar.rename(columns={"발전소명": "발전소명", "사업개시용량": "설비용량(KW)", "설치장소": "위치"}, inplace=True)

# 위도 / 경도 추가
gb_solar["위도"] = [129.0736592,128.7316020,128.5980041,128.1383791,128.6374363]
gb_solar["경도"] = [35.9595092,35.7801541,36.2453381,36.7009351,36.8854929]

# 경북 발전소 위치 및 용량
gb_solar

# 엑셀 파일 저장
# gb_solar.to_excel("dataset/경북_설비목록.xlsx", index=False, engine="openpyxl")

,발전소명,설비용량(KW),위치,위도,경도
0,고경태양광발전소,3286.35,영천시 고경면 삼귀리 산26,129.073659,35.959509
1,제2수송단태양광발전소,3091.20,경산시 남천면 신석리 178 외27필지,128.731602,35.780154
2,순수햇빛발전(주)태양광발전소,2998.80,"군위군 군위읍 상곡리 산42, 산42-6, 산42-7",128.598004,36.245338
3,해토태양광발전소,2998.80,"문경시 마성면 외어리 산80-1, 산78, 546-3, 546-4",128.138379,36.700935
4,(주)씨그마이티엔,2998.44,영주시 조와동 산49-2,128.637436,36.885493


In [28]:
# 충남지역 발전소 현황

cn_solar = pd.read_csv("dataset/충청남도_발전사업 현황.csv", encoding="cp949")

# 설비용량 높은 순서대로 정렬
mask = cn_solar.iloc[:,-1] == "태양광"
cn_solar = cn_solar[mask].reset_index(drop=True)

# 데이터 간소화
cn_solar = cn_solar[[" 발전소명 ", "설치장소", " 허가용량(키로와트) "]]

# 데이터 정렬
cn_solar = cn_solar.sort_values([" 허가용량(키로와트) "], ascending=False).reset_index(drop=True)
cn_solar = cn_solar.iloc[[0,1,2,27,28],:].reset_index(drop=True)

# 위도 / 경도 추가
cn_solar["위도"] = [126.7547192,126.5105981,126.3309592,126.6998657,126.3935127]
cn_solar["경도"] = [36.9781090,37.0216299,36.6114134,36.7963187,36.8640524]

# 충남 발전소 위치 및 용량
cn_solar

# 엑셀 파일 저장
# cn_solar.to_excel("dataset/충남_설비목록.xlsx", index=False, engine="openpyxl")

,발전소명,설치장소,허가용량(키로와트),위도,경도
0,동부당진솔라㈜,당진시 송악읍 고대리 333-2,3004.00,126.754719,36.978109
1,한전쏠라13-3호 태양광발전소,"당진시 석문면 교로리 1932-1, 1933, 1934, 1937, 1938, 19...",2999.99,126.510598,37.021630
2,당암발전10호,태안군 남면 당암리 908-4 일대 공유수면(10구역),2999.98,126.330959,36.611413
3,쎈태양광 3호 발전소,"예산군 고덕면 상몽리 1037, 1038, 1039 (3필지) / 건물 위",2999.70,126.699866,36.796319
4,연화 1호 태양광발전소,"서산시 지곡면 연화리 794, 795, 796, 797-2, 798-2, 799,...",2999.68,126.393513,36.864052
